In [ ]:
import model_functions as mf

import pandas as pd
from keras_tuner.tuners import RandomSearch
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard


PBP_DF = pd.read_csv("data/play_by_play_parsed.csv")

In [ ]:
def build_model(hidden_layers: int, layer_nodes: int, activation: str, dropout: bool, learning_rate: float):
    model = keras.Sequential()

    for _ in range(hidden_layers):
        model.add(keras.layers.Dense(
            units=layer_nodes,
            activation=activation,
        ))
        if dropout:
            model.add(keras.layers.Dropout(rate=0.20))
    model.add(keras.layers.Dense(
        units=1,
        activation='sigmoid'
    ))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
(train_X, train_y), (test_X, test_y) = mf.split_data(PBP_DF, balance=True, id_info=False, normalize=True, standardize=True)

In [ ]:
trials = [
    [6, 3, 32, "tanh", False, 0.000631579],
    [63, 3, 256, "relu", True, 0.00018857],
    [7, 3, 256, "relu", False, 0.000308675],
    [68, 3, 32, "relu", False, 0.000740252],
    [29, 3, 128, "tanh", False, 0.000714723],
    [4, 2, 256, "relu", True, 0.000314922],
    [22, 2, 256, "tanh", False, 0.000742564],
    [57, 3, 32, "relu", False, 0.001560377],
    [55, 2, 128, "relu", False, 0.000898488],
    [48, 1, 256, "relu", True, 0.000745962],
    [37, 2, 32, "relu", False, 0.000168035],
    [51, 3, 8, "tanh", False, 0.000804969],
    [54, 2, 128, "relu", False, 0.003374956],
    [11, 1, 32, "relu", True, 0.000124955],
    [15, 1, 128, "tanh", False, 0.001768874],
    [35, 2, 256, "tanh", False, 0.002197915],
    [60, 3, 32, "relu", True, 0.001227368],
]

models = {}
for trial in trials:
    name = f"{trial[0]}.5_HL-{trial[1]}_LN-{trial[2]}_{trial[3]}_{trial[4]}_LR-{trial[5]}"
    model = build_model(
        trial[1],
        trial[2],
        trial[3],
        trial[4],
        trial[5],
    )
    model.fit(train_X, train_y, validation_data=(test_X, test_y), batch_size=32, epochs=200, callbacks=[TensorBoard(f"logs/5-TTT/{name}")] )
    models[trial[0]] = model
    model.save(f"models/5-TTT-{trial[0]}.h5")

In [ ]:
model = keras.models.load_model("models/5-TTT-11.h5")
name = "11.5_HL-1_LN-32_relu_True_LR-0.000124955"
model.fit(train_X, train_y, validation_data=(test_X, test_y), batch_size=32, initial_epoch=203, epochs=300, callbacks=[TensorBoard(f"logs/5-TTT/{name}")] )